In [94]:
import copy

class Estado(object):
    
    
    def __init__(self, nome, transicoes, eh_final=False, eh_inicial=False):
        
        assert type(nome) == str
        assert type(transicoes) == dict
        assert type(eh_final) == bool
        assert type(eh_inicial) == bool
        
        self.nome = nome
        self.eh_final = eh_final
        self.eh_inicial = eh_inicial
        self.transicoes = transicoes # as chaves representam o input lido e as chaves são os nomes dos estados ao ler o input
    
    
    def entrada(self, valor):
        assert type(valor) == str
        return self.transicoes[valor]
    
    def __str__(self):
        return self.nome + str(self.transicoes)
    
    def __repr__(self):
        return self.nome
    
    def __eq__(self,other):
        assert type(other) == Estado
        return self.nome == other.nome and self.transicoes == other.transicoes and (self.eh_final == other.eh_final) and (self.eh_inicial == other.eh_inicial)
    
    def deepcopy(self):
        return Estado(self.nome, self.transicoes,self.eh_final,self.eh_inicial)
        
    
class AFD(object):
    
    
    def __init__(self, estados, estado_inicial, alfabeto):
        
        assert type(estados) == dict
        all(isinstance(e, Estado) for e in estados)
        assert type(estado_inicial) == Estado
        assert estado_inicial.eh_inicial
        assert estado_inicial == estados[estado_inicial.nome]
        assert type(alfabeto) == list
        all(isinstance(s, str) for s in alfabeto)
        
        self.estado_inicial = estado_inicial
        self.estados = estados
        self.alfabeto = alfabeto
    
    
    def entrada(self,valor):
        
        assert type(valor) == str
        
        q_atual = self.estado_inicial
        for caracter in valor:
            q_atual = self.estados[q_atual.entrada(caracter)] # Troca o estado atual pelo prox estado dado o input atual
        
        if q_atual.eh_final:
            print('Entrada aceita!')
        else:
            print('Entrada rejeitada!')
        print('Estado final: ',q_atual)
        
        return q_atual.deepcopy()
    
    
    def __str__(self):
        ans = 'AFD'
        for q in self.estados.values():
            ans += '\n'
            ans += str(q)
        return ans
    
    def equivalente(self, estado1, estado2):
        
        assert type(estado1) == Estado and type(estado2) == Estado
        
        if estado1.eh_final != estado2.eh_final:
            return False
        elif estado1.transicoes == estado2.transicoes:
            return True
        else:
            ans = {}
            for a in self.alfabeto:
                ans[a] = (estado1.transicoes[a],estado2.transicoes[a])
            return ans
    
    def compare(self,tup1,tup2):
        
        assert type(tup1) == tuple
        assert type(tup2) == tuple
        assert len(tup1) == 2
        assert len(tup2) == 2
        
        return (tup1[0] == tup2[0] and tup1[1] == tup2[1]) or (tup1[0] == tup2[1] and tup1[1] == tup2[0])
    
    def estados_equivalentes(self):
        
        verif = []
        equiv = []
        nequiv = []
        pendentes = []
        
        for e1 in self.estados.values():
            for e2 in self.estados.values():
                if e1 == e2 or e2 in verif:
                    continue
                else:
                    res = self.equivalente(e1,e2)
                    if type(res) == bool:
                        if res:
                            equiv.append((e1.nome,e2.nome))
                        else:
                            nequiv.append((e1.nome,e2.nome))
                    else:
                        pendentes.append((e1.nome,e2.nome,res))
            verif.append(e1)
        
        copy_p = copy.deepcopy(pendentes)
        copy_n = copy.deepcopy(nequiv)
        
        for p in copy_p:
            
            naoequivalente = False
            for a in self.alfabeto:
                if naoequivalente:
                    break
                for ne in copy_n:
                    if self.compare(p[2][a],ne):
                        pendentes.remove(p)
                        nequiv.append((p[0],p[1]))
                        naoequivalente = True
                        break
        
        copy_p = copy.deepcopy(pendentes)
        for p in copy_p:
            equiv.append((p[0],p[1]))
            pendentes.remove(p)
        
        return {'equivalentes':equiv, 'nao_equivalentes':nequiv}
    

In [95]:

q0 = Estado('q0',{'0':'q1','1':'q2'},eh_inicial=True,eh_final=False)
q1 = Estado('q1',{'0':'q3','1':'q4'},eh_inicial=False,eh_final=False)
q2 = Estado('q2',{'0':'q3','1':'q4'},eh_inicial=False,eh_final=False)
q3 = Estado('q3',{'0':'q1','1':'q4'},eh_inicial=False,eh_final=False)
q4 = Estado('q4',{'0':'q4','1':'q4'},eh_inicial=False,eh_final=True)

estados = {'q0':q0,'q1':q1,'q2':q2,'q3':q3,'q4':q4}

afd = AFD(estados,q0,['0','1'])

In [32]:
afd.str_estados_array(estados.values())

'[q0, q1, q2, q3, q4]'

In [4]:
print(afd)

AFD
q0{'0': 'q1', '1': 'q2'}
q1{'0': 'q3', '1': 'q4'}
q2{'0': 'q3', '1': 'q4'}
q3{'0': 'q1', '1': 'q4'}
q4{'0': 'q4', '1': 'q4'}


In [5]:
print(afd.entrada(''))

Entrada rejeitada!
Estado final:  q0{'0': 'q1', '1': 'q2'}
q0{'0': 'q1', '1': 'q2'}


In [96]:
afd.estados_equivalentes()

Equivalentes:  [('q1', 'q2'), ('q1', 'q3'), ('q2', 'q3')]
Nao equivalentes:  [('q0', 'q4'), ('q1', 'q4'), ('q2', 'q4'), ('q3', 'q4'), ('q0', 'q1'), ('q0', 'q2'), ('q0', 'q3')]
Pendentes: [
]
